## PD3.1.1
### Write a program that performs our little marble experiment. The program should allow the user to enter a Boolean matrix that describes the ways that marbles move. Make sure that the matrix follows our requirement. The user should also be permitted to enter a starting state of how many marbles are on each vertex. Then the user enters how many time clicks she wants to proceed. The computer should then calculate and output the state of the system after those time clicks. We will make changes to this program later in the chapter.

#### Solution — marble experiment (Python)

Below is a complete Python program that implements Programming Drill 3.1.1.
It:

Accepts a square Boolean matrix A (0/1 entries) that describes how marbles move.

Validates the matrix against the usual requirement used in the book: each column must have exactly one 1 (i.e., every marble on a vertex moves to exactly one vertex next tick).

Accepts a starting state vector (non-negative integers — number of marbles on each vertex).

Accepts a non-negative integer t (number of time clicks).

Computes the state after t clicks using fast exponentiation of the matrix (so it is efficient for large t).

Prints the final state; optionally prints intermediate states if you want to see the evolution.

In [ ]:
#!/usr/bin/env python3
"""
Marble experiment simulator.

Matrix A is n x n with entries 0 or 1.
Requirement: each column of A must sum to exactly 1 (each marble moves to exactly one vertex).
State is a vector of non-negative integers of length n.
State evolves as: state_next = A * state  (integer arithmetic)
"""

from typing import List

def read_int(prompt: str) -> int:
    while True:
        try:
            v = int(input(prompt).strip())
            return v
        except Exception:
            print("Please enter a valid integer.")

def read_matrix(n: int) -> List[List[int]]:
    print(f"Enter the matrix rows one by one. Each row should have {n} entries (0 or 1), separated by spaces.")
    A = []
    for i in range(n):
        while True:
            line = input(f"Row {i} : ").strip()
            parts = line.split()
            if len(parts) != n:
                print(f"Expected {n} entries; got {len(parts)}. Try again.")
                continue
            try:
                row = [int(x) for x in parts]
            except ValueError:
                print("Only integers 0 or 1 are allowed. Try again.")
                continue
            if any(x not in (0,1) for x in row):
                print("Entries must be 0 or 1. Try again.")
                continue
            A.append(row)
            break
    return A

def validate_matrix(A: List[List[int]]) -> bool:
    n = len(A)
    # check square
    if any(len(row) != n for row in A):
        print("Matrix is not square.")
        return False
    # check each column sums to exactly 1
    for j in range(n):
        col_sum = sum(A[i][j] for i in range(n))
        if col_sum != 1:
            print(f"Column {j} sums to {col_sum} (require exactly 1).")
            return False
    return True

def print_matrix(A: List[List[int]]):
    print("\nMatrix A:")
    for row in A:
        print(" ".join(str(x) for x in row))
    print()

def read_state(n: int) -> List[int]:
    while True:
        line = input(f"Enter starting state (space-separated {n} non-negative integers): ").strip()
        parts = line.split()
        if len(parts) != n:
            print(f"Expected {n} entries; got {len(parts)}. Try again.")
            continue
        try:
            state = [int(x) for x in parts]
        except ValueError:
            print("All entries must be integers. Try again.")
            continue
        if any(x < 0 for x in state):
            print("State entries must be non-negative. Try again.")
            continue
        return state

def mat_mult(A: List[List[int]], B: List[List[int]]) -> List[List[int]]:
    n = len(A)
    C = [[0]*n for _ in range(n)]
    for i in range(n):
        for k in range(n):
            if A[i][k] == 0:
                continue
            a = A[i][k]
            rowk = A[i]  # not used for micro-optim but keep structure
        for j in range(n):
            s = 0
            for k in range(n):
                s += A[i][k] * B[k][j]
            C[i][j] = s
    return C

def mat_vec_mult(A: List[List[int]], v: List[int]) -> List[int]:
    n = len(A)
    res = [0]*n
    for i in range(n):
        s = 0
        row = A[i]
        for j in range(n):
            s += row[j] * v[j]
        res[i] = s
    return res

def mat_pow(A: List[List[int]], exponent: int) -> List[List[int]]:
    """Fast exponentiation (A^exponent). A is n x n."""
    n = len(A)
    # identity
    result = [[1 if i==j else 0 for j in range(n)] for i in range(n)]
    base = [row[:] for row in A]
    e = exponent
    while e > 0:
        if e & 1:
            result = mat_mult(result, base)
        base = mat_mult(base, base)
        e >>= 1
    return result

def print_state(state: List[int]):
    print("State:", " ".join(str(x) for x in state))

def main():
    print("Marble experiment simulator")
    n = read_int("Enter number of vertices (n): ")
    if n <= 0:
        print("n must be positive.")
        return
    A = read_matrix(n)
    print_matrix(A)  # <<<<<<<<<< New Feature: print matrix after entry
    if not validate_matrix(A):
        print("Matrix validation failed. Each column must sum to exactly 1 and entries must be 0 or 1.")
        return
    state0 = read_state(n)
    t = read_int("Enter number of time clicks (non-negative integer): ")
    if t < 0:
        print("Number of time clicks must be non-negative.")
        return

    # If you want to print intermediate states, uncomment the block below:
    print("t = 0 ", end=""); print_state(state0)
    current = state0[:]
    for step in range(1, t+1):
        current = mat_vec_mult(A, current)
        print(f"t = {step} ", end=""); print_state(current)

    # Efficient computation using matrix exponentiation:
    if t == 0:
        final_state = state0
    else:
        A_t = mat_pow(A, t)
        final_state = mat_vec_mult(A_t, state0)

    print(f"\nState after {t} time clicks:")
    print_state(final_state)

if __name__ == "__main__":
    main()


Marble experiment simulator


Enter the matrix rows one by one. Each row should have 4 entries (0 or 1), separated by spaces.

Matrix A:
1 0 0 0
0 0 1 0
0 1 0 0
0 0 0 1

t = 0 State: 0 1 0 0
t = 1 State: 0 0 1 0
t = 2 State: 0 1 0 0
t = 3 State: 0 0 1 0
t = 4 State: 0 1 0 0
t = 5 State: 0 0 1 0

Matrix A:
1 0 0 0
0 0 1 0
0 1 0 0
0 0 0 1


State after 5 time clicks:
State: 0 0 1 0
